### Wide Residual Net 29 4x64
- Training Dataset: RandAugment, N=2, M=20
- Sagemaker Notebook must be of type, conda_pytorch_p36

In [1]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

#### Install Requirements

In [2]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
# Need to add this to requirements.txt
!pip install tensorboard

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/wrn.yaml \
    model.wrn.depth 28 \
    model.wrn.widening_factor 10 \
    train.batch_size 128 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_2_20 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00 \
    scheduler.epochs 400

[2020-06-21 22:06:55] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck
    

In [ ]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR

[2020-06-23 21:22:10] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: wrn
  init_mode: kaiming_fan_in
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: 

In [5]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50/test_results_0050_cifar10

[2020-06-25 22:12:09] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:27<00:00,  2.89it/s]
[2020-06-25 22:12:37] __main__ INFO: Elapsed 27.31
[2020-06-25 22:12:37] __main__ INFO: Loss 0.2885 Accuracy 0.9315


In [6]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50/test_results_0050_cifar101

[2020-06-25 22:14:31] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:05<00:00,  2.77it/s]
[2020-06-25 22:14:38] __main__ INFO: Elapsed 5.77
[2020-06-25 22:14:38] __main__ INFO: Loss 0.6531 Accuracy 0.8470


In [7]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/test_results_0400_cifar10

[2020-06-25 22:16:42] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:27<00:00,  2.90it/s]
[2020-06-25 22:17:11] __main__ INFO: Elapsed 27.29
[2020-06-25 22:17:11] __main__ INFO: Loss 0.6982 Accuracy 0.8337


In [8]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# write the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/wrn.yaml \
   model.resnet.depth 32 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/checkpoint_00400.pth \
   dataset.name CIFAR101 \
   test.output_dir /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/test_results_0400_cifar101

[2020-06-25 22:18:26] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:05<00:00,  2.78it/s]
[2020-06-25 22:18:33] __main__ INFO: Elapsed 5.75
[2020-06-25 22:18:33] __main__ INFO: Loss 1.3437 Accuracy 0.6940


In [9]:
import pandas as pd
a = pd.Series(['wrn_28_10_ra_2_20', 400, 'cifar10', 1.6982, 0.8337]) #Loss 0.6982 Accuracy 0.8337
#b = pd.Series(['wrn_28_10_ra_2_20', 300, 'cifar10', 1.0633, 0.8138])
c = pd.Series(['wrn_28_10_ra_2_20', 400, 'cifar10.1', 1.3437, 0.6940]) #Loss 1.3437 Accuracy 0.6940
#d = pd.Series(['wrn_28_10_ra_2_20', 300, 'cifar10.1',  1.9379, 0.6705])


e = pd.Series(['wrn_28_10_ra_2_20_refined400', 50, 'cifar10.1', 0.6531, 0.8470]) #Loss 0.6531 Accuracy 0.8470
f = pd.Series(['wrn_28_10_ra_2_20_refined400', 50, 'cifar10', 0.2885,0.9315]) #Loss 0.2885 Accuracy 0.9315
#g = pd.Series(['wrn_28_10_ra_2_20_refined300', 150, 'cifar10', 0.4499, 0.8795])
#h = pd.Series(['wrn_28_10_ra_2_20_refined300', 150, 'cifar10.1', 0.8206, 0.7710])
               
df_results = pd.concat([a,c,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.9 if row[2] == 'cifar10' else 89.7), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.5, 96.3) if row[2] == 'cifar10' else (88.3, 91.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,wrn_28_10_ra_2_20,400,cifar10,1.6982,0.8337,95.9,"(95.5, 96.3)"
1,wrn_28_10_ra_2_20,400,cifar10.1,1.3437,0.694,89.7,"(88.3, 91.0)"
2,wrn_28_10_ra_2_20_refined400,50,cifar10.1,0.6531,0.847,89.7,"(88.3, 91.0)"
3,wrn_28_10_ra_2_20_refined400,50,cifar10,0.2885,0.9315,95.9,"(95.5, 96.3)"


In [8]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20/exp00/test_results_0160/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[ -7.153804  ,  -0.1832159 ,  -0.69570637, ...,  -0.50926757,
         -5.526208  , -12.987257  ],
       [  2.862379  ,   7.963458  ,  -6.603018  , ...,  -4.740323  ,
         25.90399   ,  -0.52988565],
       [  4.25749   ,   8.408992  ,  -4.3299227 , ...,  -2.3715498 ,
         13.468082  ,   4.5792727 ],
       ...,
       [ -4.7270765 ,  -1.2400844 ,   1.3852903 , ...,  -0.51062894,
         -3.399443  ,  -2.4969094 ],
       [ -2.7640457 ,  14.635863  ,   6.7449965 , ...,  -1.3011913 ,
         -3.036379  ,  -7.061736  ],
       [ -2.6933427 ,   1.8961854 ,  -3.6396854 , ...,  18.63456   ,
         -2.7524152 ,  -3.2204888 ]], dtype=float32)

In [118]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/wrn_28_10_ra_2_20'
path = '/home/ec2-user/SageMaker/experiments/wrn_28_10_ra_2_20'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)